In [ ]:
source("Main.R")
source("Conf.R")
source("Utilities.R")
library(repr)
library("ggpubr")
library(plyr)


In [ ]:
setwd(projectDir)

In [ ]:
allWeights <- read.csv(par_guide_testres_file, stringsAsFactors = F)
allWeights <- reshape(allWeights[,c("guides","respGene","coef")], 
                      idvar = "guides", 
                      timevar = "respGene", 
                      direction = "wide")
rownames(allWeights) = allWeights$guides
allWeights$guides = NULL

In [ ]:
M = as.data.frame(cor(t(allWeights), method = "pearson"))
myDist <- unlist(M)
myDistFlat = data.frame(guidea=names(myDist), 
                        sDist= myDist)
myDistFlat$guidea = rep(rownames(M), each=nrow(M))
myDistFlat$guideb = rep(rownames(M), times = nrow(M))


In [ ]:
head(myDistFlat)

In [ ]:
myDistFlat <- myDistFlat[myDistFlat$guidea != myDistFlat$guideb,]
myDistFlat$guides_ab = apply(myDistFlat[,c("guidea","guideb")], 
                             1,
                             function(x){return(paste(sort(x), collapse = "_"))})

myDistFlat$guidea <- NULL
myDistFlat$guideb <- NULL

myDistFlat <- unique(myDistFlat)

myDistFlat$GeneA <- sapply(myDistFlat$guides_ab, 
                           function(x){strsplit(x,"_")[[1]][1]})
myDistFlat$GeneB <- sapply(myDistFlat$guides_ab, 
                           function(x){strsplit(x,"_")[[1]][3]})

myDistFlat$sameGene <- FALSE
myDistFlat[myDistFlat$GeneA == myDistFlat$GeneB,"sameGene"] <- TRUE

myDistFlat[myDistFlat$sameGene == "TRUE","sameGene"] = "SAME_GENE"
myDistFlat[myDistFlat$sameGene == "FALSE","sameGene"] = "DIFFERENT_GENE"


In [ ]:
allControlWeights <- read.csv(par_control_testres_file, stringsAsFactors = F)
allControlWeights <- reshape(allControlWeights[,c("guides","respGene","coef")], 
                             idvar = "guides", 
                             timevar = "respGene", 
                             direction = "wide")
rownames(allControlWeights) = allControlWeights$guides
allControlWeights$guides = NULL

In [ ]:
M_control = cor(t(allWeights), t(allControlWeights), method = "pearson")
M_control <- as.data.frame(M_control)
M_control$TargetGuide = rownames(M_control)
myDist <- melt(M_control)


In [ ]:
myDist$GeneA = sapply(myDist$variable, 
                      function(x){k = strsplit(as.character(x),"_")[[1]]
                                         return(paste0(k[-length(k)], collapse="_"))})
myDist$GeneB = "Target"
myDist$sameGene = "NO_TARGET_CONTROL"
myDist[myDist$GeneA == "ONE_NONGENE_SITE", "sameGene"] = "ONE_NONGENE_SITE_CONTROL"
myDist$guides_ab = paste0(myDist$GeneA, "_", myDist$GeneB)

myDist <- myDist[,c("value", "guides_ab", "GeneA", "GeneB", "sameGene")]
colnames(myDist) = c("sDist", "guides_ab", "GeneA", "GeneB", "sameGene")

In [ ]:
myDistFlatALL <- rbind(myDistFlat, myDist)

In [ ]:
mu <- ddply(myDistFlatALL, "sameGene", summarise, grp.mean=mean(sDist))
mu

In [ ]:
options(repr.plot.width=10, repr.plot.height=4)

ggplot(myDistFlatALL, aes(sDist, colour = sameGene)) +
  stat_ecdf(geom = "step")+theme_minimal()+
labs(
     x="Pearson correlation between guide effect sizes", 
     y = "CDF")+theme(axis.text = element_text(size=15),
              axis.title =  element_text(size=16))+xlim(-0.25, 0.25)

In [ ]:
mu <- ddply(myDistFlatALL, "sameGene", summarise, grp.mean=mean(sDist))
options(repr.plot.width=10, repr.plot.height=5)


ggplot(myDistFlatALL, aes(x=sDist, color=sameGene, fill=sameGene)) +
geom_histogram(aes(y=..density..), position="identity", alpha=0.3, binwidth = 0.02)+
geom_vline(data=mu, 
           aes(xintercept=grp.mean, color=sameGene),
           linetype="dashed")+
labs(x="Pearson correlation between guide effect sizes", 
     y = "Density")+
theme_bw()+theme(axis.text = element_text(size=15),
              axis.title =  element_text(size=16)) 

In [ ]:
x <- myDistFlatALL[myDistFlatALL$sameGene == "SAME_GENE", "sDist"]
y <- myDistFlatALL[myDistFlatALL$sameGene == "DIFFERENT_GENE", "sDist"]
ks.test(x,y, alternative="l")

In [ ]:
head(myDistFlatALL)

In order to define a threshold for guide filtering, we use linear discriminant analysis which is simply the mean of the means of the scores for the two groups.

In [ ]:
library(MASS)
library(dplyr)
tmp = myDistFlatALL[myDistFlatALL$sameGene != "DIFFERENT_GENE",
                    c("sDist", "sameGene")]
tmp[tmp$sameGene != "SAME_GENE", "sameGene"] = "CONTROL"

## sample equal number of examples to prevent the class imbalance bias
k = min(table(tmp$sameGene))
tmp = rbind(sample_n(tmp[tmp$sameGene == "SAME_GENE",],k),
            sample_n(tmp[tmp$sameGene == "CONTROL",],k))

In [ ]:
ldaFit = lda(sameGene ~ sDist, data=tmp)
plot(ldaFit)
df.pred <- predict(ldaFit, data.frame(sDist=tmp$sDist))
tmp$lda_predicted = df.pred$class 
min(tmp[tmp$lda_predicted == "SAME_GENE","sDist"])

In [ ]:
mu <- ddply(tmp, "sameGene", summarise, grp.mean=mean(sDist))
mu

In [ ]:
thresholdForGuideSelection = mean(mu$grp.mean)
thresholdForGuideSelection

In [ ]:
myDistFlatSameG = myDistFlat[myDistFlat$sameGene == "SAME_GENE", ]
myDistFlatSplit <- split(myDistFlatSameG, f = myDistFlatSameG$GeneA )

myRet <- lapply(myDistFlatSplit, function(x){ if (all(x[,"sDist"] > thresholdForGuideSelection))
                                        { 
                                           return(x)
                                     }else{
                                        x[which.max(x[,"sDist"]),]
                                   }})

myRet <- do.call(rbind, myRet)
myRet <- myRet[myRet$sDist > 0,]

myRet$guides_a <- sapply(myRet$guides_ab, function(x){kk= strsplit(x,"_")[[1]]
                                    paste0(kk[1], "_",kk[2])})

myRet$guides_b <- sapply(myRet$guides_ab, function(x){kk= strsplit(x,"_")[[1]]
                                    paste0(kk[3], "_",kk[4])})

In [ ]:
write.csv(myRet[,c("GeneA", "guides_a", "guides_b")], 
          file=par_good_KO_guides_file,
          row.names=FALSE,
          quote=FALSE)


selectedGuides = unique(c(myRet$guides_a, myRet$guides_b))
badGuides <- rownames(allWeights)[rownames(allWeights) %ni% selectedGuides]
badGuides <- sort(badGuides)
write.csv(badGuides, file=par_bad_KO_guides_file,
          quote=FALSE,
          row.names=FALSE)